In [1]:
# The values for K are is derived from the formula:  abs(sin(i+ 1))×(2**32)
K = [
0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee,
0xf57c0faf, 0x4787c62a, 0xa8304613, 0xfd469501,
0x698098d8, 0x8b44f7af, 0xffff5bb1, 0x895cd7be,
0x6b901122, 0xfd987193, 0xa679438e, 0x49b40821,
0xf61e2562, 0xc040b340, 0x265e5a51, 0xe9b6c7aa,
0xd62f105d, 0x02441453, 0xd8a1e681, 0xe7d3fbc8,
0x21e1cde6, 0xc33707d6, 0xf4d50d87, 0x455a14ed,
0xa9e3e905, 0xfcefa3f8, 0x676f02d9, 0x8d2a4c8a,
0xfffa3942, 0x8771f681, 0x6d9d6122, 0xfde5380c,
0xa4beea44, 0x4bdecfa9, 0xf6bb4b60, 0xbebfbc70,
0x289b7ec6, 0xeaa127fa, 0xd4ef3085, 0x04881d05,
0xd9d4d039, 0xe6db99e5, 0x1fa27cf8, 0xc4ac5665,
0xf4292244, 0x432aff97, 0xab9423a7, 0xfc93a039,
0x655b59c3, 0x8f0ccc92, 0xffeff47d, 0x85845dd1,
0x6fa87e4f, 0xfe2ce6e0, 0xa3014314, 0x4e0811a1,
0xf7537e82, 0xbd3af235, 0x2ad7d2bb, 0xeb86d391
]

In [2]:
# number of shifts
shift = [
    7, 12, 17, 22,
    7, 12, 17, 22,
    7, 12, 17, 22,
    7, 12, 17, 22,
    5, 9, 14, 20,
    5, 9, 14, 20,
    5, 9, 14, 20,
    5, 9, 14, 20,
    4, 11, 16, 23,
    4, 11, 16, 23,
    4, 11, 16, 23,
    4, 11, 16, 23,
    6, 10, 15, 21,
    6, 10, 15, 21,
    6, 10, 15, 21,
    6, 10, 15, 21
]

The MD5  F, G, H and I functions

In [3]:
def F(B, C, D):
    return (B & C) | ((~B) & D)


In [4]:
def G(B,C,D):
    return (B & D)| (C & (~D))

In [5]:
def H(B, C, D):
    return B ^ C ^ D

In [6]:
def I(B, C, D):
    return C ^ (B | (~D))

Modular addition

In [7]:
def mod_add(X,Y):
    Z=0x100000000
    return (X+Y)%Z

Left rotate

In [8]:
def left_rotate(x, amount):
    x &= 0xFFFFFFFF
    return ((x<<amount) | (x>>(32-amount))) & 0xFFFFFFFF

Round 
###### m_index=[
######  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
######    [1,6,11,0,5,10,15,4,9,14,3,8,13,2,7,12],
######    [5,8,11,14,1,4,7,10,13,0,3,6,9,12,15,2],
######    [0,7,14,5,12,3,10,1,8,15,6,13,4,11,2,9]
###### ]
    

In [9]:
def round(A, B, C, D,M, round_num):
    for i in range(16):
        if round_num == 1:
            fun_val = F(B, C, D)
            m_index = i
        elif round_num == 2:
            fun_val = G(B, C, D)
            m_index = (5*i + 1) % 16
        elif round_num == 3:
            fun_val = H(B, C, D)
            m_index = (3*i + 2) % 16
        else: 
            fun_val = I(B, C, D)
            m_index = (7*i) % 16

        fun_val = mod_add(fun_val, A)
        fun_val = mod_add(fun_val, K[i + 16*(round_num-1)])
        fun_val = mod_add(fun_val, int.from_bytes(M[m_index*4:(m_index+1)*4], byteorder='little'))  #???

        new_B = mod_add(B , left_rotate(fun_val, shift[i + 16*(round_num-1)])) 
        A, B, C, D = D, new_B, B, C

    return A, B, C, D

implement MD5 algorithm

In [10]:
def MD5(message):
    # Convert the string to binary
    bin_string = ''.join(format(ord(c), '08b') for c in message)
    
    # Add padding
    bin_string += '1'
    len_bits = len(bin_string)
    padding_needed = (448 - len_bits) % 512
    bin_string += '0' * padding_needed
    
    # Add length
    original_length = format(len(message) * 8, '064b')
    bin_string += original_length

    padded_message = bin_string

    # Convert the binary string to bytes
    message_bytes = int(padded_message, 2).to_bytes(len(padded_message) // 8, byteorder='little')

    # initialization vectors 
    A =0x01234567
    B =0x89abcdef
    C =0xfedcba98
    D =0x76543210
  

    # Process each 512-bit chunk
    for i in range(0, len(message_bytes), 64):
        M = message_bytes[i:i+64]
        A_temp, B_temp, C_temp, D_temp = A, B, C, D
        for round_num in range(1, 5):
            A_temp, B_temp, C_temp, D_temp = round(A_temp, B_temp, C_temp, D_temp, M, round_num)
            A = mod_add(A,A_temp)
            B = mod_add(B,B_temp)
            C = mod_add(C,C_temp)
            D = mod_add(D,D_temp)
           
    
    return (A.to_bytes(4, byteorder='little') +
            B.to_bytes(4, byteorder='little') +
            C.to_bytes(4, byteorder='little') +
            D.to_bytes(4, byteorder='little')).hex()


Round1 

In [11]:
print(MD5('They are deterministic'))
    

b4f856ba28e0f321c8ac403aeed06b14


In [12]:
import hashlib

# Input string
str = 'They are deterministic'

# Encode the string
str = str.encode()

# Calculate MD5 hash
hash_object = hashlib.md5(str)

# Get the hexadecimal representation of the hash
hash_value = hash_object.hexdigest()

print(hash_value)


23db6982caef9e9152f1a5b2589e6ca3
